# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузите и подготовьте данные. Поясните порядок действий.

In [1]:
# Импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from numpy.random import RandomState
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import warnings
warnings.simplefilter('ignore')

In [2]:
# Открываем файлы с данными
geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# Первые 5 строк
display(geo_1.head())
display(geo_2.head())
display(geo_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
# Информация
display(geo_1.info())
display(geo_2.info())
display(geo_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
# Размеры
display(geo_1.shape)
display(geo_2.shape)
display(geo_3.shape)

(100000, 5)

(100000, 5)

(100000, 5)

In [6]:
#Проверим на наличие дубликатов
display(geo_1.duplicated().sum())
display(geo_2.duplicated().sum())
display(geo_3.duplicated().sum())

0

0

0

Признак 'id' не значимый, для обучения лучше будет, если мы его удалим

In [7]:
# Удалим столбец 'id'
geo_1.drop('id', axis=1, inplace=True)
geo_2.drop('id', axis=1, inplace=True)
geo_3.drop('id', axis=1, inplace=True)

Дано: 3 набора данных со 100 000 наблюдений и 5 признаками. Признак 'id' был признан не важным и удален. Пропусков в данных не обнаружено. Дубликатов не обнаружено. Типы данных гармоничны.

## Обучите и проверьте модель для каждого региона:

2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [8]:
# Обозначим признаки и целевой признак в каждом регионе
features_1 = geo_1.drop(['product'], axis=1)
target_1 = geo_1['product']
features_2 = geo_2.drop(['product'], axis=1)
target_2 = geo_2['product']
features_3 = geo_3.drop(['product'], axis=1)
target_3 = geo_3['product']

In [9]:
# Разделим выборки на обучающую и валидационную в каждом регионе
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=20202021)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=20202021)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3, test_size=0.25, random_state=20202021)

In [10]:
# Проверим
print('Соотношения:')
print('обучающая, регион 1:', round(features_train_1.shape[0] / features_1.shape[0], 2))
print('валидационная, регион 1:', round(features_valid_1.shape[0] / features_1.shape[0], 2))
print('обучающая, регион 2:', round(features_train_2.shape[0] / features_2.shape[0], 2))
print('валидационная, регион 2:', round(features_valid_2.shape[0] / features_2.shape[0], 2))
print('обучающая, регион 3:', round(features_train_3.shape[0] / features_3.shape[0], 2))
print('валидационная, регион 3:', round(features_valid_3.shape[0] / features_3.shape[0], 2))
print()

Соотношения:
обучающая, регион 1: 0.75
валидационная, регион 1: 0.25
обучающая, регион 2: 0.75
валидационная, регион 2: 0.25
обучающая, регион 3: 0.75
валидационная, регион 3: 0.25



2.2. Обучите модель и сделайте предсказания на валидационной выборке.

2.3. Сохраните предсказания и правильные ответы на валидационной выборке.

2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [11]:
# Функция обучения, предсказания модели и расчет корня из среднеквадратичной ошибки: 
# на входе- обучающая и валидационная выборки, на выходе- предсказания модели и rmse
def predict_stock(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    print('RMSE = {:.2f}'.format(mean_squared_error(target_valid, predicted_valid) ** 0.5))
    print('Средний запас предсказанного сырья = {:.2f} рублей'.format(predicted_valid.mean()))
    return predicted_valid 

In [12]:
# Обучение модели и ее предсказания среднего запаса сырья, расчет rmse, регион 1 
geo1_pedict = predict_stock(features_train_1, features_valid_1, target_train_1, target_valid_1)

RMSE = 37.89
Средний запас предсказанного сырья = 92.58 рублей


In [13]:
# Обучение модели и ее предсказания среднего запаса сырья, расчет rmse, регион 2 
geo2_pedict = predict_stock(features_train_2, features_valid_2, target_train_2, target_valid_2)

RMSE = 0.89
Средний запас предсказанного сырья = 68.79 рублей


In [14]:
# Обучение модели и ее предсказания среднего запаса сырья, расчет rmse, регион 3 
geo3_pedict = predict_stock(features_train_3, features_valid_3, target_train_3, target_valid_3)

RMSE = 39.86
Средний запас предсказанного сырья = 95.09 рублей


2.5. Проанализируйте результаты.

Наибольшее значение среднего запаса сырья модель предсказала для региона 3, и в то же время там больше всего значение rmse, наименьший средний запас сырья по предсказанию модели в регионе 2 и в то же время там наименьшее значение rmse. 

## Подготовьтесь к расчёту прибыли:

Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно   предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения         выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы   продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше   2.5%. Среди них выбирают регион с наибольшей средней прибылью.


3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.

In [15]:
# Обозначим переменные
SAMPLE_WELLS = 500 #выборка из 500 скважин в регионе
BEST_WELLS = 200 #лучшие 200 скважин в регионе
BUDGET = 10000000000 #бюджет на разработку 1 региона
PROFIT_1 = 450000 #доход с 1000 баррелей

3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [16]:
# Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
min_stock = BUDGET / PROFIT_1 / BEST_WELLS
print('Достаточный объём сырья для безубыточной разработки новой скважины = {:.2f} рублей'.format(min_stock))

Достаточный объём сырья для безубыточной разработки новой скважины = 111.11 рублей


3.3. Напишите выводы по этапу подготовки расчёта прибыли.

Необходимый минимальный объем сырья для беубыточной разработки новой скважины составляет 111.11 тысяч баррелей. Средний предсказанный объем сырья в скважине в каждом регионе не дотягивает даже до отметки 100 000 баррелей. Но, по условиям задачи, нам необходимо выбрать 200 наилучших по объему сырья скважин. 

## Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

4.1. Выберите скважины с максимальными значениями предсказаний.

4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.

4.3. Рассчитайте прибыль для полученного объёма сырья.

In [17]:
# Функция для расчета прибыли: 
# на вход получает - список с ответами(целевой признак), предсказания модели, количество скважин,
# на выходе - возвращает прибыль
def get_revenue(target, prediction, count):
    probs_sorted = prediction.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    revenue = PROFIT_1 * selected.sum()
    return round(revenue - BUDGET, 2)

In [18]:
# применение функции, регион 1
geo1_pedict = pd.Series(geo1_pedict)
geo1_revenue = get_revenue(target_valid_1, geo1_pedict, 200)
print('Прибыль для полученного объема сырья в регионе 1 = {:.2f} рублей'.format(geo1_revenue))

Прибыль для полученного объема сырья в регионе 1 = -7836838642.33 рублей


In [19]:
# применение функции, регион 2
geo2_pedict = pd.Series(geo2_pedict)
geo2_revenue = get_revenue(target_valid_2, geo2_pedict, 200)
print('Прибыль для полученного объема сырья в регионе 2 = {:.2f} рублей'.format(geo2_revenue))

Прибыль для полученного объема сырья в регионе 2 = -8645038834.67 рублей


In [20]:
# применение функции, регион 3
geo3_pedict = pd.Series(geo3_pedict)
geo3_revenue = get_revenue(target_valid_3, geo3_pedict, 200)
print('Прибыль для полученного объема сырья в регионе 3 = {:.2f} рублей'.format(geo3_revenue))

Прибыль для полученного объема сырья в регионе 3 = -8015186066.80 рублей


На данном этапе более привелекательным выглядит регион 1.

## Посчитайте риски и прибыль для каждого региона:

5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [23]:
# Напишем функцию техника бутстреп, на вход список с ответами(целевой признак) и предсказания модели,
# для расчета средней прибыли, доверительного интервала и риска убытков.
state = RandomState(20202021)

def bootstrap(target, prediction):
    target = target.reset_index(drop=True)
    values = []
    loss = 0
    for i in range(1000):
        predicted_subsample = prediction.sample(n=500, replace=True, random_state=state)
        target_subsample = target[predicted_subsample.index]
        profit = get_revenue(target_subsample, predicted_subsample, BEST_WELLS)
        values.append(profit)
        if profit < 0:
            loss += 1
        
    values = pd.Series(values)
    
    mean = values.mean()

    lower = values.quantile(.025)
    upper = values.quantile(.975)
    
    risk_of_loss = loss / 1000
    
    print("Средняя прибыль =: {:.2f} рублей".format(mean))
    print("95 % доверительный интервал: ({:.2f},{:.2f})".format(lower, upper))
    print("Риск убытков: {0:.0%}".format(risk_of_loss))

In [24]:
print("В регионе 1")
bootstrap(target_valid_1, geo1_pedict)
print('-----------------------------------')
print("В регионе 2")
bootstrap(target_valid_2, geo2_pedict)
print('-----------------------------------')
print("В регионе 3")
bootstrap(target_valid_3, geo3_pedict)

В регионе 1
0      3.256593e+08
1      6.322177e+08
2     -5.840247e+07
3      7.628355e+08
4      4.509592e+08
           ...     
995    4.213601e+08
996    2.070292e+08
997    5.439701e+08
998    5.076281e+08
999    9.495028e+07
Length: 1000, dtype: float64
Средняя прибыль =: 410257291.10 рублей
95 % доверительный интервал: (-140381455.27,949061564.71)
Риск убытков: 8%
-----------------------------------
В регионе 2
0      7.666667e+08
1      1.616489e+08
2      5.661967e+08
3      5.047432e+08
4      6.210569e+08
           ...     
995    3.507364e+08
996    3.150337e+08
997    8.529379e+08
998    5.140732e+08
999    5.347857e+08
Length: 1000, dtype: float64
Средняя прибыль =: 487369252.76 рублей
95 % доверительный интервал: (59515260.48,930127880.07)
Риск убытков: 2%
-----------------------------------
В регионе 3
0      3.805890e+08
1      2.295803e+08
2      5.502605e+08
3      6.719055e+07
4      7.283964e+08
           ...     
995    4.830780e+08
996    6.879725e+08
997    2

5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Во втором регионе наименьший риск убытков и наибольшая прибыль, с этой точки зрения этот регион выглядит привлекательнее для разработки по сравнению с двумя остальными регионами, в которых и прибыль меньше и риск убытков больше.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

-Спасибо за качественно сделанный проект, было приятно проверять. 

- Видно, что приложено много усилий

- Код написан хорошо 

- Соблюдена структура проекта 
    
- Также оставил пару замечаний и советов
    
Удачи в следующие проектах!
</div>